In [7]:
import tensorflow as tf
from model import build_model, mix_loss, ssim_metric
try:
    model = tf.keras.models.load_model("model/model_solarcnn", 
                                    custom_objects={'mix_loss': mix_loss, 'ssim_metric': ssim_metric})
    print("model loaded")
except:
    print("model loading error")


model loaded


In [8]:
from astropy.io import fits
import os

def load_data(path):
    data_row = [(name, fits.open(os.path.join(path, name))[0].data) for name in os.listdir(path)]
    data_sort = sorted(data_row, key=lambda x: x[0])
    data = [item[1] for item in data_sort]
    name = [item[0] for item in data_sort]
    return tf.Variable(initial_value=data, dtype=tf.float32), name

try:
    test_input, test_name = load_data("data/image/test/")
    print("data loaded, total sample:", len(test_name))
except:
    print("data loading error")

data loaded, total sample: 3


In [9]:
output_path = "data/pred/"
for i in range(test_input.shape[0]):
    print("Predict sample "+str(i+1))
    test_sample = tf.reshape(test_input[i], (-1, 256, 256, 1))
    prediction = model.predict(test_sample/2000.)
    pred_name = "enh_mdi_" + test_name[i][4:20] + "SolarCNN.fits"
    fits_data = fits.PrimaryHDU(prediction[0]*2000.)
    fits_data.writeto(os.path.join(output_path, pred_name), overwrite=True)
print("Finished")

Predict sample 1
1/1 [==============================] - 1s 847ms/step
Predict sample 2
1/1 [==============================] - 0s 497ms/step
Predict sample 3
1/1 [==============================] - 0s 492ms/step
Finished
